<a href="https://colab.research.google.com/github/Makarov1999/colab/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fitdecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 2.0 MB/s eta 0:00:00


In [ ]:
file = open('/content/sample_data/2_5303101742206956492.fit', 'r')

In [ ]:
record = ['latitude', 'longitude', 'altitude', 'timestamp', 'heart_rate', 'cadence', 'speed', 'distance']

In [ ]:
!pip install folium

In [ ]:
import fitdecode
import math

In [ ]:
import pandas as pd

In [ ]:
import os

In [ ]:
from datetime import datetime, timedelta

In [ ]:
from typing import Dict, Union, Optional, Tuple





In [ ]:
def get_fit_point_data(frame: fitdecode.records.FitDataMessage) -> Optional[Dict[str, Union[float, int, str, datetime]]]:
  data: Dict[str, Union[float, int, str, datetime, None]] = {}
  if not ((frame.has_field('position_lat')) and frame.has_field('position_long')):
    return None
  if frame.get_value('position_lat') == 0 or frame.get_value('position_long') == 0:
    return None
  #if math.isnan(frame.get_value('position_lat')) or math.isnan(frame.get_value('position_long')):
    #return None
  if (frame.get_value('position_lat') is None) or (frame.get_value('position_long') is None):
    return None
  if (math.isnan(float(frame.get_value('position_lat'))) or math.isnan(float(frame.get_value('position_long')))):
    return None
  else:
    data['latitude'] = float(frame.get_value('position_lat')) / ((2**32) / 360)
    data['longitude'] = float(frame.get_value('position_long')) / ((2**32) / 360)
  for field in record[2:]:
    if (frame.has_field(field)):
      data[field] = frame.get_value(field)
      print(data)
  return data


In [ ]:
def get_fit_other_data(col, frame: fitdecode.records.FitDataMessage) -> Optional[Dict[str, Union[float, int, str, datetime]]]:
  data: Dict[str, Union[float, int, str, datetime]] = {}
  for field in col:
    data: Dict[str, Union[float, int, str, datetime]] = {}
  return data



In [ ]:
record_data = []
with fitdecode.FitReader('/content/sample_data/2_5303101742206956492.fit') as fit_file:
  for frame in fit_file:
    if (isinstance(frame, fitdecode.records.FitDataMessage)):
      if (frame.name == 'record'):
        single_point_data = get_fit_point_data(frame)
        if (single_point_data is not None):
          record_data.append(single_point_data)
record_df = pd.DataFrame(record_data, columns=record)
start_point = record_data[0]
end_point = record_data[len(record_data) - 1]


/usr/local/lib/python3.10/dist-packages/fitdecode/reader.py:909: UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"' (local_mesg_num: 0; chunk_offset: 194); adding dummy dev data...
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/fitdecode/reader.py:909: UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"' (local_mesg_num: 0; chunk_offset: 236); adding dummy dev data...
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/fitdecode/reader.py:909: UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"' (local_mesg_num: 0; chunk_offset: 287); adding dummy dev data...
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/fitdecode/reader.py:909: UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"' (local_mesg_num: 0; chunk_offset: 328); adding dummy dev data...
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packa

Выходные данные были обрезаны до нескольких последних строк (5000).
{'latitude': 59.93631734512746, 'longitude': 30.287438230589032, 'altitude': 5.0, 'timestamp': datetime.datetime(2023, 6, 11, 19, 2, 20, tzinfo=datetime.timezone.utc)}
{'latitude': 59.93631734512746, 'longitude': 30.287438230589032, 'altitude': 5.0, 'timestamp': datetime.datetime(2023, 6, 11, 19, 2, 20, tzinfo=datetime.timezone.utc), 'heart_rate': 172}
{'latitude': 59.93631734512746, 'longitude': 30.287438230589032, 'altitude': 5.0, 'timestamp': datetime.datetime(2023, 6, 11, 19, 2, 20, tzinfo=datetime.timezone.utc), 'heart_rate': 172, 'cadence': 78}
{'latitude': 59.93631734512746, 'longitude': 30.287438230589032, 'altitude': 5.0, 'timestamp': datetime.datetime(2023, 6, 11, 19, 2, 20, tzinfo=datetime.timezone.utc), 'heart_rate': 172, 'cadence': 78, 'speed': 2.753}
{'latitude': 59.93631734512746, 'longitude': 30.287438230589032, 'altitude': 5.0, 'timestamp': datetime.datetime(2023, 6, 11, 19, 2, 20, tzinfo=datetime.time

/usr/local/lib/python3.10/dist-packages/fitdecode/reader.py:909: UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"' (local_mesg_num: 0; chunk_offset: 131566); adding dummy dev data...
  warnings.warn(msg)


In [ ]:
from pyproj import CRS

In [ ]:
import geopandas as gpd
from shapely.geometry import LineString


In [ ]:
line = gpd.GeoDataFrame(crs = CRS('EPSG:4326'), geometry=[LineString(zip(record_df.longitude, record_df.latitude))])
print(line)
line_centroid = line.centroid
line_bounds = line.bounds

                                            geometry
0  LINESTRING (30.33664 59.95399, 30.33663 59.953...


<ipython-input-92-4b151029ac9a>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  line_centroid = line.centroid


In [ ]:
import folium

In [ ]:
m = folium.Map([line_centroid.y, line_centroid.x], tiles='cartodb positron')

folium.GeoJson(line).add_to(m)
folium.CircleMarker([start_point['latitude'], start_point['longitude']], radius=5,
    popup='Un cercle',
    color="#e74c3c",       # rouge
    fill_color="#27ae60",  # vert
    fill_opacity=0.9).add_to(m)
folium.CircleMarker([end_point['latitude'], end_point['longitude']], radius=5,
    popup='Un cercle',
    color="red",       # rouge
    fill_color="blue",  # vert
    fill_opacity=0.9).add_to(m)
folium.FitBounds([[line_bounds.miny[0], line_bounds.minx[0]], [line_bounds.maxy[0], line_bounds.maxx[0]]]).add_to(m)

In [ ]:
m

In [ ]:
import altair as alt
alt.Chart(record_df).mark_point().encode(y='cadence', x='distance', color='heart_rate').interactive()

alt.Chart(...)